In [ ]:
pip install requests beautifulsoup4

In [ ]:
pip install textblob

In [ ]:
pip install Textstat

In [ ]:
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
import requests
from textblob import TextBlob
from nltk.tokenize import sent_tokenize, word_tokenize
from textstat import syllable_count, textstat
from nltk.tag import pos_tag
import pyphen
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from textblob import TextBlob
import pyphen
import pandas as pd


def analyze_text_from_url(url_id,url):
    def extract_text_from_url(url):                                             # Function to extract text from a given URL
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup.get_text()
        else:
            print(f"Error: Unable to fetch content from the URL. Status code: {response.status_code}")
            return None

    def remove_stop_words(text):                                                # Function to remove stop words from the text
        words = nltk.word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)

    def count_syllables(word):                                                  # Function to count syllables in a word using Pyphen
        return len(dic.inserted(word).split('-'))

    extracted_text = extract_text_from_url(url)                                 # Extract text from the URL

    if extracted_text:
        text_without_stopwords = remove_stop_words(extracted_text)              # Remove stop words from the extracted text

        blob = TextBlob(text_without_stopwords)                                 # Create a TextBlob object

        sentiment_score = blob.sentiment.polarity                               # Sentiment analysis
        positive_score = max(sentiment_score, 0)
        negative_score = -min(sentiment_score, 0)

        sentences = sent_tokenize(text_without_stopwords)                       # Tokenize the text into sentences and words
        words = nltk.word_tokenize(text_without_stopwords)

        # Calculate various metrics
        polarity_score = blob.sentiment.polarity
        subjectivity_score = blob.sentiment.subjectivity
        average_sentence_length = sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)
        count_multi_syllable_words = sum(count_syllables(word) > 1 for word in words)
        count_complex_words = sum(count_syllables(word) > 2 for word in words)
        fog_index = 0.4 * (average_sentence_length + (count_complex_words / len(sentences)) * 100)
        word_numbers = len(words)
        per_complex_words = (count_complex_words / word_numbers) * 100
        word_counts_in_sentences = [len(word_tokenize(sentence)) for sentence in sentences]
        average_words_per_sentence = sum(word_counts_in_sentences) / len(sentences)
        pos_tags = pos_tag(words)
        personal_pronoun_tags = ['I', 'we', 'my', 'ours', 'us']
        personal_pronouns_count = sum(1 for word, pos_tag in pos_tags if pos_tag in personal_pronoun_tags)
        total_characters = sum(len(word) for word in words)
        average_word_length = total_characters / len(words)

        syllables_per_word = [count_syllables(word) for word in words]          # Calculate syllables per word using Pyphen
        average_syllables_per_word = sum(syllables_per_word) / len(words)

        # Print the results
        print(f"Positive Score: {positive_score}")
        print(f"Negative Score: {negative_score}")
        print(f"Polarity Score: {polarity_score}")
        print(f"Subjectivity Score: {subjectivity_score}")
        print(f"Average Sentence Length: {average_sentence_length} words")
        print(f"Percentage of complex words: {per_complex_words}")
        print(f"Fog Index: {fog_index:.2f}")
        print(f"Average Number of Words Per Sentence: {average_words_per_sentence:.2f}")
        print(f"Number of complex words: {count_multi_syllable_words}")
        print(f"Number of Words: {word_numbers}")
        print(f"Average Syllables Per Word: {average_syllables_per_word:.2f}")
        print(f"Number of Personal Pronouns: {personal_pronouns_count}")
        print(f"Average Word Length: {average_word_length:.2f}")

        analysis_data = {
            'URL_ID':[url_id],
            'URL': [url],
            'Positive Score': [positive_score],
            'Negative Score': [negative_score],
            'Polarity Score': [polarity_score],
            'Subjectivity Score': [subjectivity_score],
            'Average Sentence Length': [average_sentence_length],
            'Percentage of Complex Words': [per_complex_words],
            'Fog Index': [fog_index],
            'Average Number of Words Per Sentence': [average_words_per_sentence],
            'Number of Complex Words': [count_multi_syllable_words],
            'Number of Words': [word_numbers],
            'Syllables Per Word': [average_syllables_per_word],
            'Number of Personal Pronouns': [personal_pronouns_count],
            'Average Word Length': [average_word_length]
        }

        excel_file='/content/drive/MyDrive/intership/Output_bc.xlsx'
        try:                                                                    # Load the existing DataFrame or create a new one
            df = pd.read_excel(excel_file)
        except FileNotFoundError:
            df = pd.DataFrame()

        df = df.append(pd.DataFrame(analysis_data), ignore_index=True)          # Append the analysis data to the DataFrame
        df.to_excel(excel_file, index=False)                                     # Write the updated DataFrame back to the Excel file

In [ ]:
url=input("Enter the url : ")
url_id=input("Enter the URL ID : ")
analyze_text_from_url(url_id,url)
df = pd.read_excel('/content/drive/MyDrive/intership/Output_bc.xlsx')
df

Enter the url : https://insights.blackcoffer.com/continued-demand-for-sustainability/
Enter the URL ID : 52768.2
Positive Score: 0.10890633608815424
Negative Score: 0
Polarity Score: 0.10890633608815424
Subjectivity Score: 0.3968010625737898
Average Sentence Length: 28.339285714285715 words
Percentage of complex words: 24.826717076244485
Fog Index: 292.76
Average Number of Words Per Sentence: 28.34
Number of complex words: 853
Number of Words: 1587
Average Syllables Per Word: 1.96
Number of Personal Pronouns: 0
Average Word Length: 6.32


<ipython-input-76-fafc01a19c0c>:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(analysis_data), ignore_index=True)


,URL_ID,URL,Positive Score,Negative Score,Polarity Score,Subjectivity Score,Average Sentence Length,Percentage of Complex Words,Fog Index,Average Number of Words Per Sentence,Number of Complex Words,Number of Words,Syllables Per Word,Number of Personal Pronouns,Average Word Length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,0.113859,0,0.113859,0.382994,19.757282,19.803440,164.407767,19.757282,1106,2035,1.876167,0,6.319410
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.128837,0,0.128837,0.512326,29.333333,23.620130,288.876190,29.333333,665,1232,1.959416,0,6.302760
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.112461,0,0.112461,0.400480,19.302326,20.843373,168.651163,19.302326,848,1660,1.876506,0,6.095783
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,0.143729,0,0.143729,0.368483,22.101266,21.248568,196.688608,22.101266,922,1746,1.878580,0,6.147766
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,0.145582,0,0.145582,0.365687,22.740260,21.073672,200.784416,22.740260,921,1751,1.873786,0,6.143918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,50921.0,https://insights.blackcoffer.com/coronavirus-i...,0.082780,0,0.082780,0.372097,29.782609,20.437956,255.391304,29.782609,651,1370,1.840146,0,5.989781
108,51382.8,https://insights.blackcoffer.com/coronavirus-i...,0.059844,0,0.059844,0.372560,31.441176,17.680075,234.929412,31.441176,1007,2138,1.751637,0,5.839102
109,51844.6,https://insights.blackcoffer.com/what-are-the-...,0.135908,0,0.135908,0.440337,23.322222,21.581706,210.662222,23.322222,1042,2099,1.838495,0,5.998571
110,52306.4,https://insights.blackcoffer.com/marketing-dri...,0.115396,0,0.115396,0.404900,23.000000,20.568562,198.430769,23.000000,824,1794,1.828874,0,5.896878


,URL_ID,URL,Positive Score,Negative Score,Polarity Score,Subjectivity Score,Average Sentence Length,Percentage of Complex Words,Fog Index,Average Number of Words Per Sentence,Number of Complex Words,Number of Words,Syllables Per Word,Number of Personal Pronouns,Average Word Length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,0.113859,0,0.113859,0.382994,19.757282,19.803440,164.407767,19.757282,1106,2035,1.876167,0,6.319410
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.128837,0,0.128837,0.512326,29.333333,23.620130,288.876190,29.333333,665,1232,1.959416,0,6.302760
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.112461,0,0.112461,0.400480,19.302326,20.843373,168.651163,19.302326,848,1660,1.876506,0,6.095783
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,0.143729,0,0.143729,0.368483,22.101266,21.248568,196.688608,22.101266,922,1746,1.878580,0,6.147766
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,0.145582,0,0.145582,0.365687,22.740260,21.073672,200.784416,22.740260,921,1751,1.873786,0,6.143918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,46764.8,https://insights.blackcoffer.com/why-scams-lik...,0.074814,0,0.074814,0.323558,22.247706,21.195876,197.522936,22.247706,1181,2425,1.816907,0,5.900206
99,47226.6,https://insights.blackcoffer.com/impact-of-cov...,0.119921,0,0.119921,0.436037,20.638554,16.870987,147.532530,20.638554,768,1713,1.732049,0,5.730298
100,47688.4,https://insights.blackcoffer.com/impact-of-cov...,0.102163,0,0.102163,0.405306,19.537736,18.686625,153.852830,19.537736,982,2071,1.775954,0,5.694833
101,48150.2,https://insights.blackcoffer.com/impact-of-cov...,0.110879,0,0.110879,0.408182,22.554348,19.132530,181.630435,22.554348,1001,2075,1.782169,0,5.860241


In [ ]:
11668
17671.4


17671.4